# 交通信号灯识别S2TLD--PP-YOLOv2
## 1.项目背景
本项目交通信号灯识别指的是对行车红、绿、黄灯的识别，识别交通信号灯有以下应用：
1. 交通信号灯保障预警
2. 闯红灯违章抓拍
3. 引导盲人过马路


##  2.数据集介绍
本项目采用AI Studio提供数据集 

[AI Studio S2TLD数据集介绍](https://aistudio.baidu.com/aistudio/datasetdetail/108686)

另外CSDN博主「小楞」做了详细信号灯标志牌公开数据集整理：
[《信号灯标志牌公开数据集整理及思考整理》](https://blog.csdn.net/qq_33270279/article/details/105127133)

以下为其对《Bosch Small Traffic Lights Dataset》整理
### 2.1.训练集：

•5093图片

•大约每2秒钟标注一次

•10756带标注的交通灯

•红绿灯平局宽度：8.6像素

•15种不同的标签（R,R_L,R_R,Y,G,G_L,G_R (遮挡与否*2),off）

•170个灯被部分遮挡

###  2.2.测试集：

•连续8334张图像

•大约以15 fps进行注释

•13486带标注的交通灯

•红绿灯平均宽度：8.5像素

•4个标签（红色，黄色，绿色，关闭）

•2088灯被部分遮挡

### 2.3.场景覆盖情况：

•市中心繁忙的街道场景

•交通密度变化的郊区多车道

•密集的走走停停的交通

•道路工程

•光照/曝光变化很大

•乌云密布的天空，有小雨

•忽隐忽现的交通信号灯

•多个可见交通灯

•可能与交通信号灯混淆的图像部分（例如，大型圆形尾灯）

### 2.4.原始图片预览
![原始图片预览](https://ai-studio-static-online.cdn.bcebos.com/3a59dc068848427481372002f2f52e85f0facb5e15ea410996f767bdf6c10231)
![原始图片预览](https://ai-studio-static-online.cdn.bcebos.com/360f7c913d01441f912fdb3c6964b56a2c436d63a68e4f3caa31e323473aa484)


## 3.模型介绍
本项目选择PP-YOLOv2进行模型训练，该模型在coco数据集上mAP及预测速度表现领先同类模型。

## PP-YOLOv2简介
PP-YOLOv2从飞桨明星检测模型PP-YOLO出发，通过增量消融方式逐步添加有助于精度提升且不增加推理耗时的模块达到进一步提升性能的目的。PPYOLOv2（R50）COCO test数据集mAP从45.9%达到了49.5%，相较v1提升了3.6个百分点，FP32 FPS高达68.9FPS，FP16 FPS高达106.5FPS，超越了YOLOv4甚至YOLOv5！而如果使用RestNet101作为骨架网络，PPYOLOv2（R101）的mAP更高达50.3%，并且比同等精度下的YOLOv5x快15.9%！.

[10分钟快速上手使用PaddleX2.0——PP-YOLOv2目标检测](https://aistudio.baidu.com/aistudio/projectdetail/3489248?contributionType=1)

[目标检测7日打卡营](https://aistudio.baidu.com/aistudio/course/introduce/1617?directly=1&shared=1)
：详细介绍PP-YOLO优化历程


## 4.数据准备与模型训练
### 4.1.数据准备
本项目采用AI Studio公开数据集[Small Traffic Light Dataset (S2TLD)](https://aistudio.baidu.com/aistudio/datasetdetail/108686)`

原始数据压缩包目录如下：

![数据目录](https://ai-studio-static-online.cdn.bcebos.com/93b1e79f0a8443238e9e6ce4c37bfc65c5954fffc07843dbb1727f7c08a6cc41)

通过如下解压命令或windows本地操作
```
    1  cd data/ 
    2  cd data108686/   
    3  unzip S2TLD.zip 
    4  unzip -oq /home/aistudio/data/data108686/S2TLDги1080x1920гй.zip
```
手动将文件目录重命名为S2TLD_1080_1920
将数据文件夹拷贝到work目录

In [ ]:
#拷贝文件并查看文件目录
!mv data/data108686/S2TLD_1080_1920/ work/
!tree work/S2TLD_1080_1920 -d

work/S2TLD_1080_1920
├── Annotations
└── JPEGImages

2 directories


In [ ]:
#去掉文件名中空格，并生成满足PP-YOLOv2要求train.txt 
data_dir = r'work/S2TLD_1080_1920'            #数据所在目录
file_list = r'work/S2TLD_1080_1920/train.txt'

import os
with open(file_list,'w') as f:
    
    for fn_img in os.listdir(data_dir+'/JPEGImages'):
        #去掉训练图片文件名中空格    
        src = data_dir+'/JPEGImages/' + fn_img
        dst = data_dir+'/JPEGImages/' + fn_img.replace(' ','')
        if os.path.isfile(src):
            os.rename(src, dst)
            
        img_path = 'JPEGImages/' + fn_img.replace(' ','')

        #去掉训练标签文件名中空格  
        fn_xml = fn_img.rstrip('.jpg') + '.xml'
        src = data_dir+'/Annotations/' + fn_xml
        dst = data_dir+'/Annotations/' + fn_xml.replace(' ','') 
        if os.path.isfile(src):   
            os.rename(src, dst)
        annotation_path = 'Annotations/' + fn_xml.replace(' ','') 
            
        #按voc格式将文件路径写到train.txt文件中    
        #print(img_path +' '+ annotation_path)
        f.write(img_path +' '+ annotation_path + '\n')

#查看文件行数，1222 data/data108686/S2TLD_1080_1920/train.txt  与图片文件个数一致
!wc -l work/S2TLD_1080_1920/train.txt

1222 work/S2TLD_1080_1920/train.txt


### 4.2.定义数据集
请提前安装好paddlex，AI Studio 安装命令如下：


In [2]:
!pip install paddlex
!pip install pycocotools

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.6 MB 11.7 MB/s            
     |████████████████████████████████| 6.8 MB 5.3 MB/s            
     |████████████████████████████████| 1.5 MB 6.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 106 kB 25.1 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 2.0 MB 72.5 MB/s            
     |████████████████████████████████| 2.7 MB 17.2 MB/s            
     |████████████████████████████████| 151 kB 10.1 MB/s            
     |████████████████████████████████| 310 kB 12.2 MB/s            
     |████████████████████████████████| 296 kB 30.7 MB/s            
     |████████████████████████████████| 50 kB 211 kB/s             
     |████████████████████████████████| 98 kB 2.4 MB/s             
     |██

In [ ]:
#定义数据
#import sys
#sys.path.append('/home/aistudio/external-libraries')
import paddlex as pdx

#标签类别文件
label_list = r'work/S2TLD_1080_1920/class.txt'

#定义训练集transforms，采用默认参数
from paddlex import transforms as T
train_transforms = T.Compose([
    T.MixupImage(mixup_epoch=-1), T.RandomDistort(),
    T.RandomExpand(im_padding_value=[123.675, 116.28, 103.53]), T.RandomCrop(),
    T.RandomHorizontalFlip(), T.BatchRandomResize(
        target_sizes=[
            320, 352, 384, 416, 448, 480, 512, 544, 576, 608, 640, 672, 704,
            736, 768
        ],
        interp='RANDOM'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#定义训练集
train_dataset = pdx.datasets.VOCDetection(data_dir, file_list, label_list,train_transforms)

#定义验证集transforms，采用默认参数
eval_transforms = T.Compose([
    T.Resize(
        target_size=640, interp='CUBIC'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#定义验证集，这里偷懒直接用训练集数据作为验证集
eval_dataset = pdx.datasets.VOCDetection(data_dir, file_list, label_list,eval_transforms)


2022-02-25 16:00:51 [INFO]	Starting to read file list from dataset...
2022-02-25 16:00:53 [INFO]	1219 samples in file work/S2TLD_1080_1920/train.txt, including 1219 positive samples and 0 negative samples.
creating index...
index created!
2022-02-25 16:00:53 [INFO]	Starting to read file list from dataset...
2022-02-25 16:00:54 [INFO]	1219 samples in file work/S2TLD_1080_1920/train.txt, including 1219 positive samples and 0 negative samples.
creating index...
index created!


### 4.3.模型训练

AI Studio提供GPU配置如下,显存32G很棒，查询命令：nvidia-smi

![AI Studio GPU配置](https://ai-studio-static-online.cdn.bcebos.com/de5d89ad5d364a6e95d2298cc4840b24415b2bc7880446bd9f90bd4be00150de)

train_batch_size=8,显存使用20G

![AI Studio GPU配置_训练中](https://ai-studio-static-online.cdn.bcebos.com/c2625cbdf14c4b5eaa718469fed03a0c20930f0fc1184e0285864e05c3791d6d)

大于8分钟运行完一个epoch,总计耗时大约80分钟




In [7]:
#采用PPYOLOv2，backbone选用ResNet50_vd_dcn
num_classes = len(train_dataset.labels)
model = pdx.det.PPYOLOv2(num_classes=num_classes, backbone='ResNet50_vd_dcn')

model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    pretrain_weights='COCO',
    learning_rate=0.005 / 12,
    warmup_steps=1000,
    warmup_start_lr=0.0,
    lr_decay_epochs=[105, 135, 150],
    save_interval_epochs=5,
    save_dir='work/output/ppyolov2_r50vd_dcn')

2022-02-25 16:06:42 [INFO]	Downloading ppyolov2_r50vd_dcn_365e_coco.pdparams from https://paddledet.bj.bcebos.com/models/ppyolov2_r50vd_dcn_365e_coco.pdparams


  0%|          | 0/322029 [00:00<?, ?KB/s]100%|██████████| 322029/322029 [00:08<00:00, 37158.32KB/s]


2022-02-25 16:06:51 [INFO]	Loading pretrained model from work/output/ppyolov2_r50vd_dcn/pretrain/ppyolov2_r50vd_dcn_365e_coco.pdparams
2022-02-25 16:06:53 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.weight doesn't match.(Pretrained: [258, 1024, 1, 1], Actual: [33, 1024, 1, 1])
2022-02-25 16:06:53 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.bias doesn't match.(Pretrained: [258], Actual: [33])
2022-02-25 16:06:53 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.weight doesn't match.(Pretrained: [258, 512, 1, 1], Actual: [33, 512, 1, 1])
2022-02-25 16:06:53 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.bias doesn't match.(Pretrained: [258], Actual: [33])
2022-02-25 16:06:53 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.2.weight doesn't match.(Pretrained: [258, 256, 1, 1], Actual: [33, 256, 1, 1])
2022-02-25 16:06:53 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.

### 4.4模型评估

预测结果

![预测结果](https://ai-studio-static-online.cdn.bcebos.com/108f10f83a184464977d213a7996d5ef310d1820d43c4147bf882d12e2e63b6e)

真实标注

![真是标注](https://ai-studio-static-online.cdn.bcebos.com/f7e1c94769c44cd986e323a33b2b45fbc9d82a7b9f0a48d29ab2c99886464ac0)

预测结果

![预测结果_夜间](https://ai-studio-static-online.cdn.bcebos.com/0dd964e9ebbb4950b3653ab328c15c079941afcb77f04f7398e52abfc4579c40)

真实标注

![真是标注_夜间](https://ai-studio-static-online.cdn.bcebos.com/e10b7db5fe6b4f8aafd3af5591e8067bc34c8adde87e4ce48886527f6f739a9d)



In [3]:
#输出一张预测结果
import paddlex as pdx
model = pdx.load_model('work/output/ppyolov2_r50vd_dcn/best_model')
image_name = r'work/S2TLD_1080_1920/JPEGImages/2020-04-0411_09_12.194260936.jpg'

result = model.predict(image_name)
pdx.det.visualize(image_name, result, threshold=0.5, save_dir='work/output/ppyolov2_r50vd_dcn')


[02-26 12:41:10 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: Depreca

2022-02-26 12:41:18 [INFO]	Model[PPYOLOv2] loaded.
2022-02-26 12:41:18 [INFO]	The visualized result is saved at work/output/ppyolov2_r50vd_dcn/visualize_2020-04-0411_09_12.194260936.jpg


## 5.项目小结
（1）本项目基于AI Studio PP-YOLOv2模型，使用coco数据集下的预训练权重，对S2TLD数据集1080X1920分辨率下1222张照片，进行快速建模验证。从训练集预测结果来看，该模型在遮挡情况、夜间预测结果表现较好。

（2）本项目只作为快验证，S2TLD数据集下还有720x1280照片4,564张，为保证模型泛化能力应当加入全部数据进行训练，另外需要独立划分验证集与测试集进行模型训练与评估。

（3）考虑到交通信号灯标记框占图片比例较小、形状相对固定，可以考虑对anchor进行重新聚类，可能进一步提高模型进度与训练预测效率。[Anchor聚类参考](https://github.com/PaddlePaddle/PaddleX/blob/develop/docs/apis/tools/anchor_clustering.md)

## 6.作者简介
作者2013年硕士毕业，硕士期间研究方向为图形学方向。2018年开始接触深度学习。

对计算机视觉、3D视觉、OCR识别方向感兴趣，之前做过图片空间分类项目。

学习过吴恩达《机器学习》、李飞飞CS231.n课程、[李宏毅课程-机器学习进阶](https://aistudio.baidu.com/aistudio/education/group/info/1979)、[目标检测七日打卡营](https://aistudio.baidu.com/aistudio/education/group/info/1617)、
本人[知乎专栏学习笔记](https://www.zhihu.com/people/bright-49-72/posts)

联系方式：113278567@qq.com

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 